In [1]:
from sqlalchemy import create_engine
from operator import itemgetter
import json
import math
import time

from trueskill import TrueSkill, Rating, rate, quality

import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=6, suppress=True)

In [2]:
env = TrueSkill(draw_probability=0.001, backend='scipy')
env

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.1%, backend='scipy')

In [3]:
# category = 'R'
race_type = 'G'
# category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [4]:
engine = create_engine('sqlite:///../data/race.db')
conn = engine.connect()

df_race_all = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(df_race_all)))

39014 races loaded!


In [8]:
df_race = df_race_all.loc[df_sql['race_type'].isin([race_type])]
print('{} {} races for {} betting!'.format(len(df_race), race_type, bet_type))

15864 G races for W betting!


In [15]:
engine = create_engine('sqlite:///../data/player.db')
conn = engine.connect()

df_players_all = pd.read_sql_table('player', conn)
print('{} players loaded!'.format(len(df_players_all)))
# print(df_players_all.tail())

120630 players loaded!
            id  race_id race_type meeting_name  race_number  \
120625  120626    50889         G   SUNDERLAND            5   
120626  120627    50889         G   SUNDERLAND            5   
120627  120628    50889         G   SUNDERLAND            5   
120628  120629    50889         G   SUNDERLAND            5   
120629  120630    50889         G   SUNDERLAND            5   

                  raced_at             name  cnt  pos  rating_m_prev  \
120625 2017-09-07 11:04:00  STUCK FOR LOLLY    2    5      32.338503   
120626 2017-09-07 11:04:00     CLUASA SUASA    1    3      25.000000   
120627 2017-09-07 11:04:00     STARCASH RAZ    1    1      25.000000   
120628 2017-09-07 11:04:00  KELLYMOUNT CIAN    2    5      19.777133   
120629 2017-09-07 11:04:00    QUIVERING JOE    2    2      34.201816   

        rating_s_prev   rating_m  rating_s  
120625       6.019346  25.033243  4.396563  
120626       8.333333  26.573778  5.144991  
120627       8.333333  36.1053

In [16]:
df_players = df_players_all.loc[df_players_all['race_type'].isin([race_type])]
print('{} {} players for {} betting!'.format(len(df_players), race_type, bet_type))

120312 G players for W betting!


In [17]:
df_race['results'] = df_race['results_data'].map(json.loads)
df_race['runners'] = df_race['runners_data'].map(json.loads)
df_race.tail()

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
38940,50659,HOVE,GBR,K,G,2017-09-27,4,DOGS TIMEFORM NOW AVAILABLE,2017-09-27 14:07:00,Paying,515,"[[6], [3], [2], []]",NaN,"[{""runnerName"": ""PROKOFIEV"", ""runnerNumber"": 1...",11.2,29.7,144.0,NaN,"[[6], [3], [2], []]","[{'runnerName': 'PROKOFIEV', 'runnerNumber': 1..."
38941,50660,HOVE,GBR,K,G,2017-09-27,10,FOLLOW US ON TWITTER @CORAL,2017-09-27 16:07:00,Paying,515,"[[3], [6], [4], []]",NaN,"[{""runnerName"": ""LILLI MART"", ""runnerNumber"": ...",13.9,41.8,86.0,NaN,"[[3], [6], [4], []]","[{'runnerName': 'LILLI MART', 'runnerNumber': ..."
38983,50761,BELLE VUE,GBR,J,G,2017-09-17,8,BELLE VUE A6,2017-09-17 11:47:00,Paying,470,"[[1], [2], [3], []]",NaN,"[{""runnerName"": ""SEAHORSE"", ""runnerNumber"": 1,...",15.1,34.4,63.3,NaN,"[[1], [2], [3], []]","[{'runnerName': 'SEAHORSE', 'runnerNumber': 1,..."
38984,50762,ALBION PARK,QLD,B,G,2017-09-18,9,UBET - THE NEW TAB,2017-09-18 11:40:00,Paying,395,"[[6], [3], [2], [4]]",NaN,"[{""runnerName"": ""SPICE BUCKET"", ""runnerNumber""...",4.2,9.8,4.9,15.9,"[[6], [3], [2], [4]]","[{'runnerName': 'SPICE BUCKET', 'runnerNumber'..."
39002,50889,SUNDERLAND,GBR,H,G,2017-09-07,5,WILLIAM HILL SUPPORTING RACING TV,2017-09-07 11:04:00,Paying,640,"[[4], [6], [2], []]",NaN,"[{""runnerName"": ""STUCK FOR LOLLY"", ""runnerNumb...",24.0,27.4,53.0,NaN,"[[4], [6], [2], []]","[{'runnerName': 'STUCK FOR LOLLY', 'runnerNumb..."


In [ ]:
def process_strategy(x):
    """run strategy, must pass runners returned back into results (it might be modified)"""
    book = []
    for _, race in df_race.iterrows():
#         print(race['num_runners'])
        runners = race['runners']
        
        # drop scratched
        runners = [r for r in runners if r['has_odds']]
        if not runners:
            continue

        try:
            runners, num_bets = strategy(runners, bet_chunk, bet_type, x)
        except Exception as e:
            print(runners)
            raise
            
        if num_bets:
            bet_results(book, runners, bet_chunk, num_bets)
            
    book_df = pd.DataFrame(book)
    return book_df

In [ ]:
def bet_results(book, runners, bet_chunk, num_bets):
    """get results from bets"""
    place_cutoff = 3 if len(runners) >= 8 else 2

    win_diff = 0
    max_diff = 0
    prob2scales = []
    prob0scales = []
    # profit start with loss of bet made
    outcome = {
        'success': 0,
        'profit': -bet_chunk,
        'num_bets': num_bets,
        'num_runners': len(runners),
    }
    for i, runner in enumerate(runners):
        diff = abs(runner['win_scaled'] - runner[prob])
        max_diff = max(max_diff, diff)
        if int(runner['finishingPosition']) == 1:
            win_diff = diff

        if runner[bet]:
            prob2scales.append(runner[prob] / runner['win_scaled'])
            prob0scales.append(runner[prob] - runner['win_scaled'])

            if bet_type == 'W':
                if int(runner['finishingPosition']) == 1:
                    outcome['success'] = 1
                    # profit will be payout added against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['win_odds']
                    outcome['profit'] += winnings
                break
            elif bet_type == 'P':
                if 1 <= int(runner['finishingPosition']) <= place_cutoff:
                    # success is how correct each placement was
                    outcome['success'] += 1 / num_bets
                    # profit will be return added and offset against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['place_odds']
                    outcome['profit'] += winnings

    outcome['max_diff'] = max_diff
    outcome['win_diff'] = win_diff
    outcome['prob2scale_avg'] = sum(prob2scales) / len(prob2scales)
    outcome['prob2scale'] = min(prob2scales)
    outcome['prob0scale'] = min(prob0scales)
    outcome['bet_chunk'] = bet_chunk
#     outcome['runners'] = runners
    
    # add odds curve
#     probs = sorted([r['win_odds'] for r in runners if runner['win_odds']])
#     p = np.polyfit(np.arange(len(probs)), probs, 2)
#     outcome['pa'] = p[0]
#     outcome['pb'] = p[1]
#     outcome['pc'] = p[2]
    
    book.append(outcome)

In [ ]:
def bet_positive_dutch(runners, bet_chunk, bet_type, x):
    """dutch betting on probability"""
    scaled = '{}_scaled'.format(bet_type)
    pred = '{}_pred'.format(bet_type)
    prob = '{}_prob'.format(bet_type)
    bet = '{}_bet'.format(bet_type)

    # sort runners from favourite to underdog
    runners.sort(key=lambda r: r.get(pred, 0), reverse=True)
    
    # start betting on all and cut off worse runner till positive outcome
    for num_bets in range(len(runners), 0, -1):

        # reset bets
        for runner in runners:
            runner[bet] = 0

        # recreate smaller pool
        pool = runners[:num_bets]
#         print('pool is {} from {} bets'.format(len(pool), num_bets))

        # all prediction values
        total_preds = sum([r[pred] for r in pool])
#         print('total preds = {:.2f}'.format(total_preds))

        # dutch for all in pool
        profits = []
        scales = []
        for runner in pool:

            # scale bet according to prediction
            runner[bet] = bet_chunk * runner[pred] / total_preds
            runner['{}_type'.format(bet)] = 'parimutuel'
#             print('bet {:.2f}'.format(runner[bet]))

            # need to check all as we scale to probs and not odds
            if bet_type == 'W':
                odds = runner['win_odds']
                scaled = runner['win_scaled']
            elif bet_type == 'P':
                odds = runner['place_odds']
                scaled = runner['place_scaled']
#             print('odds {:.2f} and scaled {:.2f}'.format(odds, scaled))
            profits.append(runner[bet] * odds - bet_chunk)
            scales.append(runner[prob] / scaled)

        ###################################################################################
        # MIN PROFIT
        ###################################################################################
        min_profit_flag = False
        min_profit = min(profits)
        if min_profit > bet_chunk * x[0]:
            min_profit_flag = True

        ###################################################################################
        # MIN SCALED
        ###################################################################################
        min_scaled_flag = False
        min_scaled = min(scales)
        if min_scaled >= x[1]:
            min_scaled_flag = True

#         print('min profit {} min scaled {}'.format(min_profit_flag, min_scaled_flag))
        if min_profit_flag and min_scaled_flag:
#             print('breaking!')
            break
    else:
#         print('no profit determined')
        return [], 0

    # put bets from pool into runners
    for p in pool:
        for r in runners:
            if r['runnerNumber'] == p['runnerNumber']:
                r[bet] = p[bet]
                r['{}_type'.format(bet)] = p['{}_type'.format(bet)]
                break

    return runners, num_bets

In [ ]:
def f(x):
    print('.', end='')
    book_df = process_strategy(dfo, bet_positive_dutch, x)
    #print(book_df)
    if not len(book_df):
        return 0
    profit = book_df.profit.sum()
#     if bet_type == 'W':
#         participation = (len(book_df) ** 2) / (len(dfo) ** 2)
#     else:
#         participation = 1
#     return profit * participation * -1
    return profit / len(dfo) * -1
    

print('optimizing', end='')
time_start = time.time()

# x0 => min profit
# x1 => min probs2scale
# fail => avg profit

x, *grid = sp.optimize.brute(f, np.s_[0:3:5j, 0.9:1.5:20j], full_output=True)

print('')
print('done: took {:.0f}s'.format((time.time() - time_start)))
# print('x: {}'.format(x))
# print('grid: {}'.format(grid))

In [ ]:
print('category: {}'.format(category))
print('bet type: {}'.format(bet_type))
print('x: {}'.format(x))
df = process_strategy(dfo, bet_positive_dutch, x)
print('${:.2f} profit per race'.format(df.profit.sum() / len(dfo)))
print('{:.0f}% of races {} / {}'.format(len(df) / len(dfo) * 100, len(df), len(dfo)))